In [4]:
!nvidia-smi

Wed Nov  4 17:06:57 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.32.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   58C    P8    10W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [5]:
#import
import tensorflow
import pickle
from os import listdir

import cv2
import matplotlib.pyplot as plt
import numpy as np
from keras.callbacks import ModelCheckpoint
from keras.layers import Input, Flatten, Dense, Dropout, Conv2D, MaxPooling2D
from keras import Model, Sequential
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from keras.utils import to_categorical
import keras as kr

kr.__version__
tensorflow.__version__

'2.3.0'

In [9]:
#load data
def load_data():
    file = open('/content/drive/My Drive/pix.data', 'rb')

    # dump information to that file
    (pixels, labels) = pickle.load(file)

    # close the file
    file.close()

    # print(pixels.shape)
    # print(labels.shape)

    return  pixels,  labels


X, y = load_data()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=50)

gray = []
for j in range(0,len(X_train)):
    gray.append(cv2.cvtColor(np.array(X_train[j]), cv2.COLOR_BGR2GRAY))

X_train = np.array(gray)

gray = []
for j in range(0,len(X_test)):
    gray.append(cv2.cvtColor(np.array(X_test[j]), cv2.COLOR_BGR2GRAY))

X_test = np.array(gray)

# lb = LabelBinarizer()
# y_train = lb.fit_transform(y_train)
# y_test = lb.transform(y_test)

len_train = len(X_train)
len_test = len(X_test)

X_train = X_train.reshape(len_train,128,128,1)
X_test = X_test.reshape(len_test,128,128,1)

y_train = y_train.reshape(len_train,)
y_test = y_test.reshape(len_test,)
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

print("X_train ", X_train.shape)
print("y_train ",y_train.shape)

print("X_test ",X_test.shape)
print("y_test ",y_test.shape)

X_train  (2123, 128, 128, 1)
y_train  (2123, 2)
X_test  (531, 128, 128, 1)
y_test  (531, 2)


In [10]:


model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape = (128,128,1), activation = 'relu'))
model.add(Conv2D(32, (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.5))
model.add(Conv2D(64, (3, 3), activation = 'relu'))
model.add(Conv2D(64, (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(units = 128, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(units = 2, activation = 'softmax'))

# initialize our initial learning rate and # of epochs to train for
INIT_LR = 0.001
EPOCHS = 100

# Compiling the CNN

# opt = kr.optimizers.rmsprop(lr=INIT_LR, decay=1e-6)
model.compile(optimizer = "adam", loss = 'categorical_crossentropy', metrics = ['accuracy'])

# Fitting the CNN to the images
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=EPOCHS, batch_size=32)

model.save("modal.h5")



Epoch 1/100
67/67 [==============================] - 3s 38ms/step - loss: 10.8940 - accuracy: 0.7687 - val_loss: 0.0590 - val_accuracy: 0.9774
Epoch 2/100
67/67 [==============================] - 2s 33ms/step - loss: 0.0354 - accuracy: 0.9887 - val_loss: 0.0081 - val_accuracy: 0.9981
Epoch 3/100
67/67 [==============================] - 2s 33ms/step - loss: 0.0240 - accuracy: 0.9911 - val_loss: 0.0212 - val_accuracy: 0.9906
Epoch 4/100
67/67 [==============================] - 2s 33ms/step - loss: 0.0367 - accuracy: 0.9892 - val_loss: 0.0260 - val_accuracy: 0.9962
Epoch 5/100
67/67 [==============================] - 2s 33ms/step - loss: 0.0216 - accuracy: 0.9925 - val_loss: 0.0250 - val_accuracy: 0.9906
Epoch 6/100
67/67 [==============================] - 2s 33ms/step - loss: 0.0165 - accuracy: 0.9939 - val_loss: 6.8797e-04 - val_accuracy: 1.0000
Epoch 7/100
67/67 [==============================] - 2s 33ms/step - loss: 0.0013 - accuracy: 1.0000 - val_loss: 2.0346e-04 - val_accuracy: 1.00